Непараметрические тесты часто основаны на рангах или медианах, а не на конкретных значениях данных. Это означает, что они могут упускать детали, которые учитываются параметрическими тестами, работающими с точными значениями и предположениями о распределении.

In [1]:
import numpy as np
from scipy import stats

# Создание данных
np.random.seed(42)  # Для воспроизводимости

# Первая выборка из нормального распределения
data1 = np.random.normal(loc=50, scale=5, size=100)

# Вторая выборка из распределения с другим масштабом (например, логнормальное распределение)
data2 = np.random.lognormal(mean=np.log(50), sigma=0.5, size=100)

# Параметрический t-тест
t_stat, p_value_ttest = stats.ttest_ind(data1, data2)
print("Параметрический t-тест:", p_value_ttest)

# Непараметрический тест Манна-Уитни
u_stat, p_value_mannwhitney = stats.mannwhitneyu(data1, data2)
print("Непараметрический тест Манна-Уитни:", p_value_mannwhitney)

# Проверка гипотез
alpha = 0.05
print("Параметрический t-тест отвергает нулевую гипотезу?" , p_value_ttest < alpha)
print("Непараметрический тест Манна-Уитни отвергает нулевую гипотезу?", p_value_mannwhitney < alpha)


Параметрический t-тест: 0.01878748881021487
Непараметрический тест Манна-Уитни: 0.2984969742960175
Параметрический t-тест отвергает нулевую гипотезу? True
Непараметрический тест Манна-Уитни отвергает нулевую гипотезу? False


In [2]:
data1.mean(), data2.mean()

(49.48076741302953, 56.96020736283199)

В данном примере непараметрический тест скорее ошибся

Но при этом когда данных мало или данные распределены не по нормальному закону t-тест не работает

In [3]:
import numpy as np
from scipy import stats

# Устанавливаем seed для воспроизводимости
np.random.seed(42)

# Первая выборка из нормального распределения
data1 = np.random.normal(loc=50, scale=5, size=100)

# Вторая выборка из нормального распределения с несколькими выбросами
data2 = np.random.normal(loc=50, scale=5, size=97)
data2 = np.append(data2, [100, 100,100])  # Добавляем выбросы

# Параметрический t-тест
t_stat, p_value_ttest = stats.ttest_ind(data1, data2)
print("Параметрический t-тест:", p_value_ttest)

# Непараметрический тест Манна-Уитни
u_stat, p_value_mannwhitney = stats.mannwhitneyu(data1, data2)
print("Непараметрический тест Манна-Уитни:", p_value_mannwhitney)

# Проверка гипотез
alpha = 0.05
print("Параметрический t-тест отвергает нулевую гипотезу?", p_value_ttest < alpha)
print("Непараметрический тест Манна-Уитни отвергает нулевую гипотезу?", p_value_mannwhitney < alpha)


Параметрический t-тест: 0.04457750981305762
Непараметрический тест Манна-Уитни: 0.2872845529734861
Параметрический t-тест отвергает нулевую гипотезу? True
Непараметрический тест Манна-Уитни отвергает нулевую гипотезу? False


In [4]:
data1.mean(), data2.mean()

(49.48076741302953, 51.65807475892157)

В этом примере скорее ошибся t-тест из-за того, что в данных выбросы и распределение не похоже на нормальное

# непараметрический тест (тест пермутаций)

Основная идея теста пермутаций:

Проверка нулевой гипотезы, которая предполагает, что наблюдаемые данные произошли из одного и того же распределения.

Генерация всех возможных (или большого количества) перестановок данных.

Вычисление статистики для каждой перестановки и сравнение ее с наблюдаемой статистикой.

Допустим, у нас есть две группы студентов, каждая из разных классов. Мы хотим проверить, отличается ли средний балл Класса 1 от Класса 2.

**Шаг 1: Сформулируем гипотезу**

Нулевая гипотеза, H0, заключается в том, что средние значения двух выборок не различаются, то есть средняя оценка в классе 1 такая же, как средняя оценка в классе 2.

Альтернативная гипотеза, HA, состоит в том, что существует разница в средних значениях двух выборок — средняя оценка в классе 1 не совпадает со средней оценкой в классе 2.

**Шаг 2: Рассчитаем наблюдаемую разницу в средних значениях**

Подсчитайте среднюю оценку для каждого класса. Затем вычислите абсолютную разницу между двумя средними значениями. Это наша наблюдаемая разница.

**Шаг 3: Объединим данные**

Объедините оценки обоих классов в один большой пул. Это связано с тем, что в соответствии с нулевой гипотезой мы говорим, что не имеет значения, учится ли учащийся в классе 1 или в классе 2 — оценки исходят от одного и того же населения.

**Шаг 4: Запустим тест перестановки**

Вот суть теста перестановки:

Перемешайте все оценки и случайным образом разделите их на две группы. Группы должны быть того же размера, что и исходные Класс 1 и Класс 2. Это моделирует нулевую гипотезу о том, что класс, в котором находится учащийся, не имеет значения.

Вычислите среднее значение каждой группы, а затем абсолютную разницу между этими двумя средними значениями.

Повторите шаги 1-2 большое количество раз (например, 10 000 раз), каждый раз записывая разницу в средствах.

**Шаг 5: Рассчитаем p-value**

Значение p представляет собой долю переставленных повторений, где абсолютная разница в средних значениях была больше или равна наблюдаемой разнице в средних значениях. Если эта пропорция (значение p) меньше установленного уровня значимости (часто выбираемого как 0,05), то мы отклоняем нулевую гипотезу.

**Вывод **

Если мы отклоним нулевую гипотезу, мы придем к выводу, что существует статистически значимая разница в средних оценках между классом 1 и классом 2. Если мы не отвергнем нулевую гипотезу, мы придем к выводу, что у нас недостаточно доказательств, чтобы сказать, что средние значения другой.

In [13]:
import random

In [14]:
# sample data
classroom_1_grades = [85, 88, 92, 78, 90, 88, 79, 91, 85, 87]
# classroom_2_grades = [20, 22, 35, 29, 21, 28, 32, 44, 39, 53]

# classroom_2_grades = [90, 88, 92, 78, 90, 88, 79, 91, 85, 87]
classroom_2_grades = [70, 72, 85, 79, 81, 78, 82, 84, 79, 83]

# function to calculate mean
def mean(lst):
    return sum(lst) / len(lst)

# observed difference
obs_diff = abs(mean(classroom_1_grades) - mean(classroom_2_grades))
print("Mean of group 1: ", mean(classroom_1_grades))
print("Mean of group 2: ", mean(classroom_2_grades))
print("Delta: ", obs_diff)

print(f'1 group = {classroom_1_grades}')
print(f'2 group = {classroom_2_grades}')


Mean of group 1:  86.3
Mean of group 2:  79.3
Delta:  7.0
1 group = [85, 88, 92, 78, 90, 88, 79, 91, 85, 87]
2 group = [70, 72, 85, 79, 81, 78, 82, 84, 79, 83]


In [16]:

# combined data
combined_grades = classroom_1_grades + classroom_2_grades
combined_grades

[85,
 88,
 92,
 78,
 90,
 88,
 79,
 91,
 85,
 87,
 70,
 72,
 85,
 79,
 81,
 78,
 82,
 84,
 79,
 83]

In [21]:
random.shuffle(combined_grades)
combined_grades

[78,
 79,
 85,
 72,
 83,
 92,
 85,
 81,
 85,
 79,
 91,
 78,
 90,
 87,
 82,
 84,
 70,
 79,
 88,
 88]

In [24]:
mean(combined_grades[:len(classroom_1_grades)])

81.9

In [25]:
mean(combined_grades[len(classroom_1_grades):])

83.7

In [27]:
abs(mean(combined_grades[len(classroom_1_grades):]) - mean(combined_grades[:len(classroom_1_grades)]))

1.7999999999999972

In [28]:

# combined data
combined_grades = classroom_1_grades + classroom_2_grades

# permuation test
num_permutations = 10000
perm_diffs = []

for _ in range(num_permutations):
    random.shuffle(combined_grades)
    perm_diffs.append(abs(mean(combined_grades[:len(classroom_1_grades)]) - mean(combined_grades[len(classroom_1_grades):])))

print("permutation deltas: ", perm_diffs[:3])


permutation deltas:  [2.799999999999997, 1.0, 2.5999999999999943]


In [29]:
perm_diffs[:10]

[2.799999999999997,
 1.0,
 2.5999999999999943,
 6.799999999999997,
 1.0,
 3.799999999999997,
 1.4000000000000057,
 3.5999999999999943,
 1.4000000000000057,
 2.200000000000003]

In [30]:
sum(perm_diff >= obs_diff for perm_diff in perm_diffs)

45

In [9]:
len(perm_diffs)

10000

In [10]:
sum(perm_diff >= obs_diff for perm_diff in perm_diffs)/len(perm_diffs)

0.0054

или тоже самое

In [11]:
from scipy.stats import permutation_test

In [12]:
#  Определение статистики для теста
def mean_diff(x, y):
    return np.mean(x) - np.mean(y)

# Проведение перестановочного теста
res = permutation_test((classroom_1_grades, classroom_2_grades), mean_diff, n_resamples=10000, alternative='two-sided')

# Вывод результатов
print(f"p-value: {res.pvalue}")
print(f"Observed statistic: {res.statistic}")

p-value: 0.0053994600539946005
Observed statistic: 7.0
